Nome: Vinicius Freitas Schiavinato Olzon


Matrícula: 20210026803


- O objetivo deste exercício é treinar e avaliar uma rede neural recorrente (RNN
ou LSTM) para classificar um conjunto de dados sobre notícias de acordo com
suas categorias.
- A rede neural pode ser criada utilizando o PyTorch ou Tensorflow/Keras. A rede
neural deve ser avaliada através do cálculo da acurácia (quantidade de acertos
dividido pela quantidade total de testes).

### Carrega as bibliotecas necessárias para puxar o dataset

In [55]:
%pip install torch torchvision torchaudio
%pip install ipywidgets
%pip install torchtext
%pip install datasets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [56]:
!pip install ipywidgets widgetsnbextension pandas-profiling

Defaulting to user installation because normal site-packages is not writeable


In [57]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [58]:
import torch
import datasets
import pandas as pd
from torch import nn

import matplotlib.pyplot as plt
from ipywidgets import FloatProgress
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [59]:
dataset = datasets.load_dataset("okite97/news-data")

In [60]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Title', 'Excerpt', 'Category'],
        num_rows: 4686
    })
    test: Dataset({
        features: ['Title', 'Excerpt', 'Category'],
        num_rows: 828
    })
})

### Tokenização

In [61]:
tokenizer = get_tokenizer('basic_english')

In [62]:
tokenizer(dataset['train']["Excerpt"][0])

['uefa',
 'has',
 'opened',
 'disciplinary',
 'proceedings',
 'against',
 'barcelona',
 ',',
 'juventus',
 'and',
 'real',
 'madrid',
 'over',
 'their',
 'involvement',
 'in',
 'the',
 'proposed',
 'european',
 'super',
 'league',
 '.']

In [63]:
tokens_list_train = []
for i in range(len(dataset['train']['Excerpt'])):
    tokens_list_train.append(tokenizer(dataset['train']['Excerpt'][i]))

In [64]:
tokens_list_test = []
for i in range(len(dataset['test']['Excerpt'])):
    tokens_list_test.append(tokenizer(dataset['test']['Excerpt'][i]))

### Adiciona as colunas novas de tokens nos dados de treino e teste

In [65]:
dataset['train'] = dataset['train'].add_column(name='tokens', column=tokens_list_train)

In [66]:
dataset['test'] = dataset['test'].add_column(name='tokens', column=tokens_list_test)

In [67]:
dataset['train']['tokens'][0]

['uefa',
 'has',
 'opened',
 'disciplinary',
 'proceedings',
 'against',
 'barcelona',
 ',',
 'juventus',
 'and',
 'real',
 'madrid',
 'over',
 'their',
 'involvement',
 'in',
 'the',
 'proposed',
 'european',
 'super',
 'league',
 '.']

In [68]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Title', 'Excerpt', 'Category', 'tokens'],
        num_rows: 4686
    })
    test: Dataset({
        features: ['Title', 'Excerpt', 'Category', 'tokens'],
        num_rows: 828
    })
})

### Vocabulário

### Tokens desconhecidos ficarão como índice 0

In [69]:
vocab_custom = build_vocab_from_iterator(dataset['train']['tokens'], min_freq=3, specials=['<unk>'])
vocab_custom.set_default_index(vocab_custom['<unk>'])

In [70]:
vocab_custom['european']

178

In [71]:
vocab_custom['<unk>']

0

In [72]:
vocab_custom['cleber']

0

In [73]:
len(vocab_custom)

4137

In [74]:
def vocab(tokens):
  result = []
  for token in tokens:
    result.append(vocab_custom[token])

  return result

In [75]:
vocab(['that', 'football', 'player', 'is', 'going', 'to', 'be', 'the', 'mvp'])

[18, 139, 549, 25, 797, 4, 44, 1, 0]

In [76]:
vocab(['that', 'football', 'player', 'is', 'going', 'to', 'be', 'the', 'best'])

[18, 139, 549, 25, 797, 4, 44, 1, 509]

### Alterando 'Category' para valores numéricos

In [77]:
categoria_numerico_treino = pd.factorize(dataset['train']['Category'])[0]
categoria_numerico_teste = pd.factorize(dataset['test']['Category'])[0]

/tmp/ipykernel_8205/937684725.py:1: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  categoria_numerico_treino = pd.factorize(dataset['train']['Category'])[0]
/tmp/ipykernel_8205/937684725.py:2: FutureWarning: factorize with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  categoria_numerico_teste = pd.factorize(dataset['test']['Category'])[0]


In [78]:
print(dataset['train']['Category'][:6])

['sports', 'business', 'politics', 'health', 'politics', 'sports']


In [79]:
print(categoria_numerico_treino[:6])

[0 1 2 3 2 0]


In [80]:
print(dataset['test']['Category'][:6])

['politics', 'politics', 'business', 'health', 'business', 'politics']


In [81]:
print(categoria_numerico_teste[:6])

[0 0 1 2 1 0]


In [82]:
dataset['train'] = dataset['train'].add_column(name="Category_id", column=categoria_numerico_treino)
dataset['test'] = dataset['test'].add_column(name="Category_id", column=categoria_numerico_teste)

In [83]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Title', 'Excerpt', 'Category', 'tokens', 'Category_id'],
        num_rows: 4686
    })
    test: Dataset({
        features: ['Title', 'Excerpt', 'Category', 'tokens', 'Category_id'],
        num_rows: 828
    })
})

In [84]:
dataset['train']['Category'][:6]

['sports', 'business', 'politics', 'health', 'politics', 'sports']

### Token to ID

In [85]:
def token2id(row):
    row['tokens_id'] = vocab(row['tokens'])
    return row

In [86]:
dataset = dataset.map(token2id)

Map:   0%|          | 0/4686 [00:00<?, ? examples/s]

Map:   0%|          | 0/828 [00:00<?, ? examples/s]

In [87]:
print(dataset['train']['tokens'][0])

['uefa', 'has', 'opened', 'disciplinary', 'proceedings', 'against', 'barcelona', ',', 'juventus', 'and', 'real', 'madrid', 'over', 'their', 'involvement', 'in', 'the', 'proposed', 'european', 'super', 'league', '.']


In [88]:
print(dataset['train']['tokens_id'][0])

[577, 7, 755, 0, 3008, 64, 229, 2, 465, 10, 210, 196, 41, 36, 3633, 6, 1, 572, 178, 123, 50, 9]


In [89]:
print(dataset['train']['Category_id'])

[0, 1, 2, 3, 2, 0, 2, 2, 2, 0, 2, 0, 1, 0, 4, 1, 1, 1, 2, 3, 1, 0, 0, 0, 2, 1, 0, 5, 1, 2, 5, 2, 2, 2, 4, 3, 2, 5, 2, 1, 2, 1, 0, 5, 2, 2, 0, 0, 2, 1, 1, 2, 2, 0, 1, 0, 4, 5, 0, 4, 1, 1, 3, 1, 1, 1, 1, 1, 2, 0, 5, 1, 1, 3, 3, 0, 1, 0, 5, 1, 5, 1, 2, 1, 1, 2, 2, 4, 0, 5, 0, 1, 1, 1, 4, 0, 2, 0, 3, 2, 1, 3, 0, 5, 5, 1, 2, 0, 1, 2, 2, 1, 4, 4, 2, 1, 3, 1, 0, 0, 0, 1, 1, 0, 0, 2, 2, 1, 1, 0, 1, 1, 2, 3, 0, 2, 0, 3, 1, 0, 2, 3, 4, 3, 1, 1, 0, 3, 1, 2, 0, 3, 2, 1, 2, 2, 0, 0, 3, 2, 1, 0, 0, 3, 3, 5, 2, 0, 3, 1, 1, 1, 2, 3, 3, 1, 1, 2, 0, 0, 2, 1, 3, 1, 1, 1, 1, 4, 3, 5, 4, 1, 0, 3, 1, 5, 0, 3, 1, 4, 0, 1, 2, 3, 2, 3, 3, 0, 5, 1, 3, 3, 1, 4, 0, 1, 1, 1, 2, 1, 1, 4, 1, 1, 0, 0, 2, 0, 1, 0, 0, 1, 3, 1, 2, 1, 3, 0, 1, 2, 0, 0, 3, 1, 4, 4, 0, 0, 0, 2, 4, 1, 2, 1, 2, 3, 0, 2, 1, 1, 1, 5, 3, 2, 2, 3, 0, 1, 2, 2, 2, 0, 0, 1, 1, 2, 1, 0, 5, 1, 1, 2, 5, 3, 0, 2, 2, 0, 0, 0, 0, 1, 2, 2, 1, 5, 0, 0, 4, 3, 3, 0, 1, 3, 2, 2, 1, 0, 2, 0, 5, 3, 1, 0, 3, 0, 0, 0, 0, 0, 3, 3, 3, 0, 3, 3, 0, 0, 2, 1, 5, 2, 1, 

In [90]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Title', 'Excerpt', 'Category', 'tokens', 'Category_id', 'tokens_id'],
        num_rows: 4686
    })
    test: Dataset({
        features: ['Title', 'Excerpt', 'Category', 'tokens', 'Category_id', 'tokens_id'],
        num_rows: 828
    })
})

In [91]:
dataset['train'].shape

(4686, 6)

### Dataloader

In [92]:
class NEWSDataset(Dataset):
  def __init__(self, dataset):
    self.data = dataset['tokens_id']
    self.labels = dataset['Category_id']

  def __len__(self):
    return len(self.data)

  def __getitem__(self, x):
    return torch.tensor(self.data[x]), torch.tensor(self.labels[x])

In [93]:
dataset_news = NEWSDataset(dataset['train'])

In [94]:
dataset_news[0][0].shape, dataset_news[0][1].shape

(torch.Size([22]), torch.Size([]))

In [95]:
embed_len = 32
hidden_dim = 16
n_layers = 1
n_classes = len(list(set(dataset['train']['Category'])))

class NewsClassifier(nn.Module):
  def __init__(self):
    super(NewsClassifier, self).__init__()

    ## embedding 4686x32
    self.embedding_layer = nn.Embedding(num_embeddings=len(vocab_custom),embedding_dim=embed_len)
    ## rnn 32x16
    self.rnn = nn.RNN(input_size=embed_len, hidden_size=hidden_dim, num_layers=n_layers)
    ## linear 16x6
    self.linear = nn.Linear(hidden_dim, n_classes)
    self.softmax = nn.Softmax(dim=0)

  def forward(self, input):
    embeddings = self.embedding_layer(input)
    final_hidden = self.rnn(embeddings)
    return self.softmax(self.linear(final_hidden[0]))
  
model = NewsClassifier()

In [96]:
print(dataset['train']['tokens_id'][0])

[577, 7, 755, 0, 3008, 64, 229, 2, 465, 10, 210, 196, 41, 36, 3633, 6, 1, 572, 178, 123, 50, 9]


In [97]:
len(dataset['train']['tokens_id'][0])

22

In [98]:
model(torch.tensor(dataset['train']['tokens_id'][0]))

tensor([[0.0457, 0.0509, 0.0354, 0.0645, 0.0495, 0.0569],
        [0.0405, 0.0552, 0.0367, 0.0438, 0.0561, 0.0475],
        [0.0634, 0.0320, 0.0600, 0.0377, 0.0419, 0.0714],
        [0.0479, 0.0149, 0.0367, 0.0563, 0.0612, 0.0407],
        [0.0219, 0.0363, 0.0512, 0.0643, 0.0470, 0.0325],
        [0.0464, 0.0376, 0.0362, 0.0690, 0.0296, 0.0443],
        [0.0485, 0.0274, 0.0620, 0.0512, 0.0302, 0.0962],
        [0.0683, 0.0611, 0.0387, 0.0423, 0.0640, 0.0277],
        [0.0529, 0.0905, 0.0410, 0.0365, 0.0331, 0.0386],
        [0.0478, 0.0345, 0.0487, 0.0339, 0.0548, 0.0392],
        [0.0393, 0.0383, 0.0250, 0.0332, 0.0437, 0.0450],
        [0.0424, 0.0857, 0.0436, 0.0550, 0.0346, 0.0356],
        [0.0618, 0.0474, 0.0338, 0.0590, 0.0742, 0.0236],
        [0.0371, 0.0336, 0.0830, 0.0322, 0.0420, 0.0582],
        [0.0574, 0.0646, 0.0428, 0.0386, 0.0591, 0.0225],
        [0.0322, 0.0925, 0.0268, 0.0682, 0.0333, 0.0478],
        [0.0369, 0.0251, 0.0514, 0.0479, 0.0435, 0.0528],
        [0.058

In [99]:
device = ("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

NewsClassifier(
  (embedding_layer): Embedding(4137, 32)
  (rnn): RNN(32, 16)
  (linear): Linear(in_features=16, out_features=6, bias=True)
  (softmax): Softmax(dim=0)
)

In [100]:
LR = 1e-3
EPOCHS = 5
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [101]:
from tqdm.notebook import tqdm

In [102]:
def train(dataloader):
    size = len(dataloader.dataset)
    train_loss = 0

    for batch, (data, label) in enumerate(dataloader):
        data, label = data.to(device), label.to(device)
        optimizer.zero_grad()
        out = model(data[0])
        label_tensor = torch.tensor(label).to(device).squeeze()

        loss = loss_fn(out, label_tensor) 

        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        if batch % 1000 == 0:
            current = batch * len(data)
            print(f' loss: {loss.item():>7f}  [{current:>5d}/{size:>5d}]')

    train_loss /= size
    return train_loss

In [103]:
dataloader_train = DataLoader(NEWSDataset(dataset['train']), shuffle=True)


In [104]:
def test(dataloader):
    size = len(dataloader.dataset)
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, label in tqdm(dataloader):
            data, label = data.to(device), label.to(device)
            out = model(data[0])
            label_tensor = torch.tensor(label).to(device).squeeze()

            loss = loss_fn(out, label_tensor)

            test_loss += loss.item()
            _, predicted = torch.max(out, 0)
            correct += (predicted == label_tensor).sum().item()

    test_loss /= size
    accuracy = correct / size

    print(f' test accuracy: {(100 * accuracy):>0.1f}%, test loss: {test_loss:>8f} ')

    return test_loss

In [105]:
dataloader_test = DataLoader(NEWSDataset(dataset['test']), shuffle=True)

In [ ]:
train_losses = []
test_losses = []
for t in range(EPOCHS):
    print(f'Epoch {t+1} -------------------------------')

    print('Train')
    train_losses.append(train(dataloader_train))

    print('Test')
    test_losses.append(test(dataloader_test))

In [ ]:
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.legend()
plt.show()